In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import urllib.request
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import pandas as pd
import os

In [3]:
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model = InceptionV3(weights='imagenet',
                                input_shape=(150, 150, 3),
                                include_top=False)

for layer in pre_trained_model.layers:
    layer.trainable = False

last_layer = pre_trained_model.output

87910968/87910968 [==============================] - 3s 0us/step


In [4]:
train_dir = '/content/drive/MyDrive/Capstone/Dataset/Tomato/train'
validation_dir = '/content/drive/MyDrive/Capstone/Dataset/Tomato/valid'

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    rotation_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest',
    horizontal_flip=True)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(150, 150),
                                                    batch_size=128,
                                                    class_mode='binary')

val_datagen = ImageDataGenerator(rescale=1. / 255)

val_generator = val_datagen.flow_from_directory(validation_dir,
                                                target_size=(150, 150),
                                                batch_size=32,
                                                class_mode='binary')

x = last_layer
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)

model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['acc'])

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


In [5]:
csv_file = '/content/drive/MyDrive/Capstone/Dataset/Tomato/train/_annotations.csv'
df = pd.read_csv(csv_file)

# Get the paths from the "class" column
input_path = df['class'].tolist()

# Create an ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Create a generator from the dataframe
train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='/content/drive/MyDrive/Capstone/Dataset/Tomato/train',
    x_col="filename",
    y_col="class",
    subset="training",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(150, 150)
)


label = []
for class_name in input_path:
    if class_name == 'halfripe':
        label.append(0)
    elif class_name == 'ripe' :
        label.append(1)
    else:
        label.append(2)

print(input_path[0], label[0])

Found 7541 validated image filenames belonging to 3 classes.
ripe 1


In [6]:
csv_file = '/content/drive/MyDrive/Capstone/Dataset/Tomato/valid/_annotations.csv'
df = pd.read_csv(csv_file)

# Get the paths from the "class" column
input_path = df['class'].tolist()

# Create an ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

val_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory='/content/drive/MyDrive/Capstone/Dataset/Tomato/valid',
    x_col="filename",
    y_col="class",
    subset="validation",
    batch_size=32,
    seed=42,
    shuffle=True,
    class_mode="categorical",
    target_size=(150, 150)
)


Found 132 validated image filenames belonging to 3 classes.


In [7]:
def solution_A3():
  model.fit(train_generator,
          epochs=5,
          steps_per_epoch=236,
          verbose=1,
          validation_data=val_generator,
          validation_steps=31)

  return model

In [8]:
if __name__ == '__main__':
  model = solution_A3()

Epoch 1/5
236/236 [==============================] - ETA: 0s - loss: 0.6608 - acc: 0.6650

236/236 [==============================] - 57s 187ms/step - loss: 0.6608 - acc: 0.6650 - val_loss: 0.6393 - val_acc: 0.6667
Epoch 2/5
236/236 [==============================] - 33s 141ms/step - loss: 0.6376 - acc: 0.6667
Epoch 3/5
236/236 [==============================] - 33s 140ms/step - loss: 0.6374 - acc: 0.6667
Epoch 4/5
236/236 [==============================] - 33s 139ms/step - loss: 0.6372 - acc: 0.6667
Epoch 5/5
236/236 [==============================] - 34s 142ms/step - loss: 0.6371 - acc: 0.6667


In [10]:
from keras.preprocessing import image
import numpy as np

img_path = '/content/download (1).jpeg'
img = image.load_img(img_path, target_size=(150, 150))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array /= 255.0

predictions = model.predict(img_array)
if predictions[0][0] > 0.5:
    label = 'halfripe'
elif predictions[0][0] <= 0.5:
    label = 'unripe'
else:
    label = 'ripe'

print(label)


1/1 [==============================] - 2s 2s/step
unripe
